We'll work with a dataset containing housing characteristics and skip the basic data set-up code.  The end result is:
- The housing characteristics for the training data are stored in a DataFrame `train_predictors`.  We'll use it to predict home prices in a Series called `target`.  
- The housing characteristics for the test data are stored in a DataFrame `test_predictors`.

In [ ]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

# Read the data
X = pd.read_csv('../input/train.csv', index_col='Id') 
X_test = pd.read_csv('../input/test.csv', index_col='Id')

# Remove rows with missing target, separate target from predictors
X.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X.SalePrice
X.drop(['SalePrice'], axis=1, inplace=True)

# Select numerical columns
numerical_cols = [cname for cname in X.columns if X[cname].dtype in ['int64', 'float64']]

# Drop numerical columns with missing values
cols_with_missing = [col for col in numerical_cols if X[col].isnull().any()] 
X.drop(cols_with_missing, axis=1, inplace=True)
X_test.drop(cols_with_missing, axis=1, inplace=True)

# Break off validation set from training data
X_train_tmp, X_valid_tmp, y_train, y_valid = train_test_split(X, y,
                                                              train_size=0.8, test_size=0.2,
                                                              random_state=0)

# Get categorical columns
object_cols = [cname for cname in X_train_tmp.columns if
               X_train_tmp[cname].dtype == "object"]

# Impute missing values 
my_imputer = SimpleImputer(strategy='most_frequent')
cat_cols_train = pd.DataFrame(my_imputer.fit_transform(X_train_tmp[object_cols]))
cat_cols_valid = pd.DataFrame(my_imputer.transform(X_valid_tmp[object_cols]))

# Imputation removed index and column names; put them back
cat_cols_train.index = X_train_tmp.index
cat_cols_valid.index = X_valid_tmp.index
cat_cols_train.columns = X_train_tmp[object_cols].columns
cat_cols_valid.columns = X_valid_tmp[object_cols].columns

# Replace categorical columns with imputed categorical columns
X_train = pd.concat([X_train_tmp.drop(object_cols, axis=1), cat_cols_train], axis=1)
X_valid = pd.concat([X_valid_tmp.drop(object_cols, axis=1), cat_cols_valid], axis=1)

In [ ]:
# get list of categorical variables
print("Categorical variables:")

for col in object_cols:
    print(col, ':', len(X_train[col].unique()))

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

In [ ]:
drop_X_train = X_train.select_dtypes(exclude=['object'])
drop_X_valid = X_valid.select_dtypes(exclude=['object'])

print("MAE from Approach 1 (Drop categorical variables):")
print(score_dataset(drop_X_train, drop_X_valid, y_train, y_valid))

In [ ]:
from sklearn.preprocessing import LabelEncoder

# make copy to avoid changing original data 
label_X_train = X_train.copy()
label_X_valid = X_valid.copy()

# apply label encoder to each column with categorical data
my_encoder = LabelEncoder()
X = pd.concat([X_train, X_valid], axis=0)
for col in object_cols:
    my_encoder.fit(X[col])
    label_X_train[col] = my_encoder.transform(X_train[col])
    label_X_valid[col] = my_encoder.transform(X_valid[col])

print("MAE from Approach 2 (Label Encoding):") 
print(score_dataset(label_X_train, label_X_valid, y_train, y_valid))

In [ ]:
# one-hot encode categorical data
OH_X_train = pd.get_dummies(X_train)
OH_X_valid = pd.get_dummies(X_valid)

# ensure columns are in same order in both datasets
OH_X_train, OH_X_valid = OH_X_train.align(OH_X_valid, join='inner', axis=1)

print("MAE from Approach 3 (One-Hot Encoding):") 
print(score_dataset(OH_X_train, OH_X_valid, y_train, y_valid))

In [ ]:
high_cardinality_cols = [cname for cname in X_train.columns if
                         X_train[cname].nunique() > 10 and
                         X_train[cname].dtype == "object"]

print(high_cardinality_cols)

In [ ]:
# one-hot encode categorical data
OH_X_train_low = pd.get_dummies(X_train.drop(high_cardinality_cols, axis=1))
OH_X_valid_low = pd.get_dummies(X_valid.drop(high_cardinality_cols, axis=1))

# ensure columns are in same order in both datasets
OH_X_train_low, OH_X_valid_low = OH_X_train_low.align(OH_X_valid_low, join='inner', axis=1)

print("MAE from Approach 3 (One-Hot Encoding):") 
print(score_dataset(OH_X_train_low, OH_X_valid_low, y_train, y_valid))